In [1]:
import os
import ast
import json


def extract_definitions(file_path):
    """Extract defined variables and functions from a Python file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        code = f.read()
    tree = ast.parse(code)
    definitions = {}

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            definitions[node.name] = {'type': 'function', 'context': code}
        elif isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name) and isinstance(target.ctx, ast.Store):
                    definitions[target.id] = {'type': 'variable', 'context': code}

    return definitions


def suggest_better_name(name, def_type, context):
    """Simulate LLM suggesting a snake_case name based on context."""
    # Placeholder: In practice, I'd analyze context and suggest a name
    # Share your code for real suggestions!
    if def_type == 'function':
        return f"process_{name.lower()}" if not name.islower() else name
    elif def_type == 'variable':
        return f"{name.lower()}_value" if not name.islower() else name
    return name  # Fallback


def generate_rename_map(directory, map_file):
    all_definitions = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                defs = extract_definitions(file_path)
                all_definitions[file_path] = defs

    rename_map = {}
    for file_path, defs in all_definitions.items():
        for name, info in defs.items():
            if name not in rename_map:  # Only suggest once per unique name
                new_name = suggest_better_name(name, info['type'], info['context'])
                if new_name != name and not new_name.startswith('_'):  # Avoid private names
                    rename_map[name] = new_name

    # Save to JSON
    with open(map_file, 'w', encoding='utf-8') as f:
        json.dump(rename_map, f, indent=4)
    print(f"Generated rename map saved to {map_file}")


if __name__ == "__main__":
    project_dir = input("Enter your project directory: ")
    map_file = "rename_map.json"
    generate_rename_map(project_dir, map_file)

Generated rename map saved to rename_map.json
